<p style="text-align: center; font-weight: bold;">
    Personalisierte Ernährung zur Prävention ernährungsbedingter Krankheiten<br>
    von <br>
    Gruppe 5 – Flavia De Solda, Kay Sheppard, Murat Ayhekim, Adrian Martic, Adrian Wegmann<br>
    FFHS/BFH/HSLU und OST – Master of Science in Wirtschaftsinformatik<br>
    März 2025
</p>

## Erklärung

Wir erklären hiermit, dass wir die vorliegende Arbeit für das Modul BINA bzw. die von uns ausgewiesene Leistung selbständig, ohne unzulässige Hilfe Dritter und ausschließlich unter Verwendung der angegebenen Quellen verfasst bzw. erbracht haben. Zur sprachlichen Überprüfung, zur besseren Verständlichkeit der Zusammenhänge sowie zur Sicherstellung eines konsistenten roten Fadens wurde unterstützend ein KI-Tool eingesetzt.

Zürich, 13.06.2025  
Die unterschriebene Selbständigkeitserklärung befindet sich im Anhang.


---

## 1 Einleitung
Im Kontext steigender chronischer Erkrankungen wie Typ-2-Diabetes, Bluthochdruck und Adipositas rückt die Frage nach präventiver, individueller Ernährung zunehmend in den Fokus. Während Ernährungsempfehlungen in der Vergangenheit vor allem pauschalisiert wurden, erlauben moderne Datenanalysen eine differenzierte Betrachtung individueller Ernährungsbedürfnisse. Ziel dieser Fallstudie ist es, mithilfe datengetriebener Methoden Erkenntnisse über individuelle Ernährungsmuster und deren Zusammenhang mit Gesundheitszuständen zu gewinnen.
Wir orientieren uns dabei am CPA-Framework "From Data to Decisions" und fokussieren uns insbesondere auf die ersten vier Schritte: 1) Zieldefinition und Informationsbedarf, 2) Datenerhebung und -integration, 3) Datenanalyse und 4) Kommunikation der Erkenntnisse für Entscheider.


## 2 Analyse
### 2.1 Haupt Ziel
### 2.1 Strategisches Ziel
Das strategische Ziel dieser Case Study ist es, herauszufinden, wie datenbasierte, personalisierte Ernährungsempfehlungen zur Reduktion ernährungsbedingter Krankheitsrisiken generiert werden können. Diese Erkenntnisse sollen unter anderem als Basis für ein potenzielles digitales Geschäftsmodell im Ernährungsbereich dienen.
### 2.2 Zielgruppen
- Gesundheitsbewusste Konsument:innen mit spezifischen Bedürfnissen
- Start-ups / Unternehmen im Bereich Digital Health & Nutrition
- Ernährungsberater:innen, die datenunterstützt Empfehlungen geben möchten
### 2.2 Vorgehen
Um das Hauptziel zu erreichen, den Informationsbedarf zu decken und die Analysefragen zu beantworten, wird in dieser Arbeit – wie in der Einleitung beschrieben – dem CPA-Vorgehen „From Data to Decisions“ gefolgt.

- **Schritt 1: Ziele und Informationsbedarf definieren**  
  Festlegen, welche strategischen Ziele verfolgt werden und welche Informationen dafür benötigt werden – wer was wann und warum wissen muss.

- **Schritt 2: Daten erheben**  
  Relevante Daten sammeln, um den Informationsbedarf zu decken. Prüfen, ob Daten bereits vorhanden sind oder wie sie am besten erhoben werden können.

- **Schritt 3: Daten analysieren**  
  Die gesammelten Daten auswerten, um daraus verwertbare Erkenntnisse zu gewinnen.

- **Schritt 4: Informationen präsentieren**  
  Die Analyseergebnisse verständlich, im richtigen Format und zur richtigen Zeit an die relevanten Personen weitergeben.

- **Schritt 5: Datenbasierte Entscheidungen treffen**  
  Informationen in fundierte Entscheidungen umsetzen – weg von Bauchgefühl, hin zu datengestütztem Handeln.

### Schritt 1: Ziele und Informationsbedarf

| KAQ Nr. | Key Analytics Question | Nächste Schritte | Mögliche Stolperfallen / Hinweise |
|--------|-------------------------|------------------|------------------------------------|
| 1 | Welche Ernährungsmuster lassen sich aus individuellen Makronährstoffprofilen ableiten – und welche Krankheitsrisiken sind damit verbunden? | - Clusteranalyse (z. B. k-Means) auf aggregierten Makros  <br> - Verknüpfung mit Krankheitsspalte | Clustervalidierung schwierig bei geringer Fallzahl; Krankheiten müssen in Dummy-Variablen umgewandelt werden |
| 2 | Wie unterscheiden sich die Nährstoffprofile zwischen Personen mit und ohne Bluthochdruck – in Bezug auf Tagesverteilung (z. B. salzreiche Mittagessen)? | - Vergleichsplots/Statistiken pro Mahlzeit  <br> - Hypothesentest (z. B. t-Test, ANOVA) | Uneinheitliche Portionsgrößen oder Zeiteinteilung könnten Verzerrung verursachen |
| 3 | Welche Lebensmittel tragen am stärksten zur Gesamtaufnahme kritischer Nährstoffe (z. B. Zucker, Natrium) bei – je nach Krankheitsgruppe? | - Mapping Mahlzeiten auf Lebensmittel (Food & Nutrition Dataset)  <br> - Summieren pro Person & Krankheitsgruppe | Mapping kann fuzzy sein; evtl. NLP-basierte Matching nötig |
| 4 | Gibt es typische Nährstoffüberschüsse/-defizite in der Ernährung von Personen mit mehreren chronischen Krankheiten? | - Subset für multimorbide Personen bilden  <br> - Abweichung von WHO-Zielen berechnen | Krankheiten müssen zuverlässig extrahiert sein (Split der Disease-Spalte) |
| 5 | Wie stark weichen die tatsächlichen Nährstoffwerte von empfohlenen Zielwerten ab – nach Aktivitätslevel, Alter oder Krankheitstyp? | - Berechnung Zielwerte pro Person mit WHO-Tabelle  <br> - Delta berechnen und visualisieren | WHO-Werte sind oft als Bereich angegeben – klare Regel für Mittelwert/Niedrig/Hoch treffen |
| 6 | Welche Kombinationen aus Ernährungspräferenz (z. B. vegan) und Makronährstoffverteilung sind mit besseren Gesundheitsindikatoren verbunden? | - Gruppierung nach "Dietary Preference"  <br> - Vergleich der Krankheitsprävalenz & Makros | Gruppengrößen evtl. unausgeglichen (z. B. wenig Veganer:innen im Sample) |
| 7 | Welche Krankheiten treten überdurchschnittlich häufig bei bestimmten Essensverteilungen (z. B. kalorienreiches Frühstück vs. Dinner) auf? | - Verhältnis Kalorien Frühstück vs. Dinner berechnen  <br> - Vergleich mit Krankheitslabels | Mahlzeiten müssen vollständig und gleichmäßig erfasst sein |
| 8 | Lässt sich ein einfacher Risiko-Score aus Ernährungsdaten ableiten, der z. B. ≥80 % der „Hypertension“-Fälle korrekt identifiziert? | - Scoring-Modell mit Schwellenwerten oder einfachem ML-Modell  <br> - Kreuzvalidierung | Überanpassung (Overfitting) bei geringer Datenmenge möglich |
| 9 | Gibt es eine kritische Schwelle bestimmter Nährstoffe (z. B. >60 g Zucker), ab der Krankheitsrisiken sprunghaft ansteigen? | - Schwellenanalyse, z. B. Entscheidungsbaum oder Visualisierung | Kausalität vs. Korrelation beachten |
| 10 | Welche Kombination aus Lebensmitteln führt (rechnerisch) am häufigsten zu Risikoprofilen? | - Reverse Engineering aus Food & Nutrition + Krankheitsdaten  <br> - Rangliste erstellen | Repräsentativität einzelner Lebensmittel evtl. nicht gegeben |


## 3 Datenmanagement & Datensammlung
### 3.1Datenquellen
#### Verwendete Datensätze

Folgende Datensätze werden in dieser Arbeit verwendet und ggf. miteinander verknüpft:

1. **"Detailed Meals Macros CLEANED" (Kaggle)**
   - Personenbezogene Merkmale (Alter, Geschlecht, Groesse, Gewicht, Aktivitaetslevel)
   - Naehrwertprofile der Mahlzeiten (Fruehstueck, Mittag, Abendessen, Snacks)
   - Krankheit(en) pro Person (z. B. Hypertension, Diabetes, Weight Gain)

2. **"Food and Nutrition" Dataset (Kaggle)**
   - Naehrwerte einzelner Lebensmittel (Makros & Mikros)
   - Erlaubt Zuweisung und Aufschluesselung komplexer Mahlzeiten

3. **BLV Schweizer Referenzwerte**
   - Ideale Naehrstoffzufuhrempfehlungen


In [ ]:
#pip install scikit-learn
#pip install kaggle

In [ ]:
# =====================================
# Schritt 0: Setup & Bibliotheken
# =====================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# =====================================
# Schritt 1: Daten laden & erste Sichtung
# =====================================

# === Päckli 1: FOOD Nutrition ===

# Lade die Datensätze aus Päckli 1
df_meals = pd.read_csv("https://raw.githubusercontent.com/M2xAy/ffhs-datensaetze/refs/heads/main/Datasets/archive/detailed_meals_macros_CLEANED.csv")
df_nutrition = pd.read_csv("https://raw.githubusercontent.com/M2xAy/ffhs-datensaetze/refs/heads/main/Datasets/archive/Food_and_Nutrition__.csv")
# Vorschau
print("Food meals:")
df_meals.info()
df_meals.head()
print("Food nutrition:")
df_nutrition.info()
df_nutrition.head()




# === Päckli 2: FOOD DATA GROUPS ===


# Lade alle Gruppen-Datensätze
food_data_files = [
    "https://raw.githubusercontent.com/M2xAy/ffhs-datensaetze/refs/heads/main/Datasets/Kaggle/FINAL%20FOOD%20DATASET/FOOD-DATA-GROUP1.csv",
      "https://raw.githubusercontent.com/M2xAy/ffhs-datensaetze/refs/heads/main/Datasets/Kaggle/FINAL%20FOOD%20DATASET/FOOD-DATA-GROUP2.csv",
        "https://raw.githubusercontent.com/M2xAy/ffhs-datensaetze/refs/heads/main/Datasets/Kaggle/FINAL%20FOOD%20DATASET/FOOD-DATA-GROUP3.csv",
          "https://raw.githubusercontent.com/M2xAy/ffhs-datensaetze/refs/heads/main/Datasets/Kaggle/FINAL%20FOOD%20DATASET/FOOD-DATA-GROUP4.csv",
            "https://raw.githubusercontent.com/M2xAy/ffhs-datensaetze/refs/heads/main/Datasets/Kaggle/FINAL%20FOOD%20DATASET/FOOD-DATA-GROUP5.csv",
]# Falls du alle in ein gemeinsames DataFrame packen willst
food_data_dfs = []
for file_url in food_data_files:
    df = pd.read_csv(file_url)
    food_data_dfs.append(df)

# Zusammenfuegen in ein einziges DataFrame
food_groups_combined = pd.concat(food_data_dfs, ignore_index=True)
# Vorschau
print("Food Groups Combined:")
food_groups_combined.info()
food_groups_combined.head()


# Lade die Nährstoff-Referenzwerte der Schweiz
blv_nutrient_requirements = pd.read_csv("https://raw.githubusercontent.com/M2xAy/ffhs-datensaetze/refs/heads/main/Datasets/BLV/Nutrient_requirements_CH-2022.csv")
# Vorschau
print("\nBLV Nutrient Requirements:")
blv_nutrient_requirements.info()
blv_nutrient_requirements.head()

Food meals:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1698 entries, 0 to 1697
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Ages                     1698 non-null   int64  
 1   Gender                   1698 non-null   object 
 2   Height                   1698 non-null   int64  
 3   Weight                   1698 non-null   int64  
 4   Activity Level           1698 non-null   object 
 5   Dietary Preference       1698 non-null   object 
 6   Daily Calorie Target     1698 non-null   int64  
 7   Protein                  1698 non-null   int64  
 8   Sugar                    1698 non-null   float64
 9   Sodium                   1698 non-null   float64
 10  Calories                 1698 non-null   int64  
 11  Carbohydrates            1698 non-null   int64  
 12  Fiber                    1698 non-null   float64
 13  Fat                      1698 non-null   int64  
 14  Breakfast Su

,Ages,Gender,Height,Weight,Activity Level,Dietary Preference,Daily Calorie Target,Protein,Sugar,Sodium,...,Dinner Protein.1,Dinner Carbohydrates.1,Dinner Fats,Snack Suggestion,Snacks Calories,Snacks Protein,Snacks Carbohydrates,Snacks Fats,Disease,Lunch Fats
0,25,Male,180,80,Moderately Active,Omnivore,2000,120,125.0,24.0,...,60.0,250.0,60.0,Greek yogurt with fruit,150,10,20,5,Weight Gain,12.0
1,32,Female,165,65,Lightly Active,Vegetarian,1600,80,100.0,16.0,...,5.0,31.0,27.0,Apple with almond butter,180,4,30,8,"Weight Gain, Hypertension, Heart Disease",8.0
2,48,Male,175,95,Sedentary,Vegan,2200,100,150.0,20.0,...,55.0,150.0,15.0,Trail mix,300,8,34,20,Weight Gain,15.0
3,55,Female,160,70,Very Active,Omnivore,2500,140,175.0,28.0,...,20.0,163.0,27.0,Banana with peanut butter,210,5,30,9,Weight Gain,10.0
4,62,Male,170,85,Sedentary,Vegetarian,2000,80,125.0,16.0,...,6.3,186.0,22.0,Fruit and nut mix,250,6,28,15,Weight Gain,9.0


Food nutrition:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1698 entries, 0 to 1697
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Ages                  1698 non-null   int64  
 1   Gender                1698 non-null   object 
 2   Height                1698 non-null   int64  
 3   Weight                1698 non-null   int64  
 4   Activity Level        1698 non-null   object 
 5   Dietary Preference    1698 non-null   object 
 6   Daily Calorie Target  1698 non-null   int64  
 7   Protein               1698 non-null   int64  
 8   Sugar                 1698 non-null   float64
 9   Sodium                1698 non-null   float64
 10  Calories              1698 non-null   int64  
 11  Carbohydrates         1698 non-null   int64  
 12  Fiber                 1698 non-null   float64
 13  Fat                   1698 non-null   int64  
 14  Breakfast Suggestion  1698 non-null   object 
 15  Lunch

,Ages,Gender,Height,Weight,Activity Level,Dietary Preference,Daily Calorie Target,Protein,Sugar,Sodium,Calories,Carbohydrates,Fiber,Fat,Breakfast Suggestion,Lunch Suggestion,Dinner Suggestion,Snack Suggestion,Disease
0,25,Male,180,80,Moderately Active,Omnivore,2000,120,125.0,24.0,2020,250,30.0,60,Oatmeal with berries and nuts,Grilled chicken salad with mixed greens,Salmon with roasted vegetables,Greek yogurt with fruit,Weight Gain
1,32,Female,165,65,Lightly Active,Vegetarian,1600,80,100.0,16.0,1480,200,24.0,40,Tofu scramble with veggies,Lentil soup with whole wheat bread,Vegetable stir-fry with brown rice,Apple with almond butter,"Weight Gain, Hypertension, Heart Disease"
2,48,Male,175,95,Sedentary,Vegan,2200,100,150.0,20.0,2185,300,36.0,65,Tofu and veggie breakfast burrito,Black bean burger on a whole wheat bun,Lentil and vegetable curry,Trail mix,Weight Gain
3,55,Female,160,70,Very Active,Omnivore,2500,140,175.0,28.0,2680,350,42.0,80,Greek yogurt with granola and fruit,Chicken and vegetable stir-fry,Turkey chili with brown rice,Banana with peanut butter,Weight Gain
4,62,Male,170,85,Sedentary,Vegetarian,2000,80,125.0,16.0,1815,250,30.0,55,Scrambled eggs with whole wheat toast and avocado,Quinoa salad with chickpeas and vegetables,Vegetarian chili with cornbread,Fruit and nut mix,Weight Gain


Food Groups Combined:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2395 entries, 0 to 2394
Data columns (total 37 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0.1          2395 non-null   int64  
 1   Unnamed: 0            2395 non-null   int64  
 2   food                  2395 non-null   object 
 3   Caloric Value         2395 non-null   int64  
 4   Fat                   2395 non-null   float64
 5   Saturated Fats        2395 non-null   float64
 6   Monounsaturated Fats  2395 non-null   float64
 7   Polyunsaturated Fats  2395 non-null   float64
 8   Carbohydrates         2395 non-null   float64
 9   Sugars                2395 non-null   float64
 10  Protein               2395 non-null   float64
 11  Dietary Fiber         2395 non-null   float64
 12  Cholesterol           2395 non-null   float64
 13  Sodium                2395 non-null   float64
 14  Water                 2395 non-null   float64
 15 

,Unnamed: 0.1,Unnamed: 0,food,Caloric Value,Fat,Saturated Fats,Monounsaturated Fats,Polyunsaturated Fats,Carbohydrates,Sugars,...,Calcium,Copper,Iron,Magnesium,Manganese,Phosphorus,Potassium,Selenium,Zinc,Nutrition Density
0,0,0,cream cheese,51,5.0,2.9,1.3,0.200,0.8,0.500,...,0.008,14.100,0.082,0.027,1.300,0.091,15.5,19.100,0.039,7.070
1,1,1,neufchatel cheese,215,19.4,10.9,4.9,0.800,3.1,2.700,...,99.500,0.034,0.100,8.500,0.088,117.300,129.2,0.054,0.700,130.100
2,2,2,requeijao cremoso light catupiry,49,3.6,2.3,0.9,0.000,0.9,3.400,...,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,5.400
3,3,3,ricotta cheese,30,2.0,1.3,0.5,0.002,1.5,0.091,...,0.097,41.200,0.097,0.096,4.000,0.024,30.8,43.800,0.035,5.196
4,4,4,cream cheese low fat,30,2.3,1.4,0.6,0.042,1.2,0.900,...,22.200,0.072,0.008,1.200,0.098,22.800,37.1,0.034,0.053,27.007


ParserError: Error tokenizing data. C error: Expected 1 fields in line 42, saw 47


In [ ]:
# =====================================
# Schritt 2: Daten bereinigen
# =====================================

In [ ]:
# =====================================
# Schritt 3: Desktripitve Statistik - Teil 1 Mittelwerte, Verteilungen, Boxplots, Gruppenvergleiche
# =====================================


# ------------------------------
# 3.1.1 Verteilung der Makronährstoffe
# ------------------------------
macro_columns = ["Calories", "Protein", "Fat", "Sugar", "Sodium", "Carbohydrates"]
fig1, axes = plt.subplots(len(macro_columns), 1, figsize=(10, 20))

for i, col in enumerate(macro_columns):
    sns.histplot(df_meals[col], kde=True, ax=axes[i], color='skyblue')
    axes[i].set_title(f"Verteilung von {col}", fontsize=12)
    axes[i].set_xlabel(col)
    axes[i].set_ylabel("Häufigkeit")

plt.tight_layout()
plt.show()

# ------------------------------
# 3.1.2 Häufigkeit von Krankheiten
# ------------------------------
plt.figure(figsize=(10, 6))
df_meals["Disease"].value_counts().plot(kind="bar", color="salmon")
plt.title("Häufigkeit von Krankheiten (kombiniert)", fontsize=13)
plt.xlabel("Krankheit(en)")
plt.ylabel("Anzahl Personen")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# ------------------------------
# 3.1.3 Vergleich Kalorienaufnahme nach Geschlecht
# ------------------------------
plt.figure(figsize=(8, 6))
sns.boxplot(data=df_meals, x="Gender", y="Calories", palette="pastel")
plt.title("Kalorienaufnahme nach Geschlecht", fontsize=13)
plt.xlabel("Geschlecht")
plt.ylabel("Kalorien")
plt.tight_layout()
plt.show()




# ------------------------------
# Deskriptive Statistik mit erweiterten Kennzahlen
# ------------------------------
columns = ["Calories", "Protein", "Fat", "Sugar", "Sodium", "Carbohydrates"]
desc_stats = df_meals[columns].describe().T  # count, mean, std, min, 25%, 50%, 75%, max
desc_stats["median"] = df_meals[columns].median()
desc_stats["skewness"] = df_meals[columns].skew()
desc_stats["kurtosis"] = df_meals[columns].kurt()

# Ausgabe als Tabelle
print("Deskriptive Statistik (inkl. Median, Schiefe & Kurtosis):\n")
print(desc_stats.round(2))

# ------------------------------
# Korrelationen (Heatmap)
# ------------------------------
correlation_matrix = df_meals[columns].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Korrelation zwischen Makronährstoffen", fontsize=13)
plt.tight_layout()
plt.show()


# ------------------------------
# Deskriptive Statistik mit erweiterten Kennzahlen
# ------------------------------
columns = ["Calories", "Protein", "Fat", "Sugar", "Sodium", "Carbohydrates"]
desc_stats = df_meals[columns].describe().T  # count, mean, std, min, 25%, 50%, 75%, max
desc_stats["median"] = df_meals[columns].median()
desc_stats["skewness"] = df_meals[columns].skew()
desc_stats["kurtosis"] = df_meals[columns].kurt()

# Ausgabe als Tabelle
print("Deskriptive Statistik (inkl. Median, Schiefe & Kurtosis):\n")
print(desc_stats.round(2))

# ------------------------------
# Korrelationen (Heatmap)
# ------------------------------
correlation_matrix = df_meals[columns].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Korrelation zwischen Makronährstoffen", fontsize=13)
plt.tight_layout()
plt.show()



In [ ]:
# =====================================
# Schritt 4: Desktripitve Statistik - Teil 2 Verteilung der Krankheiten & Ernährungsstile
# =====================================

In [ ]:
# =====================================
# Schritt 5: Clusteranalyse
# =====================================

# 🧠 Frage: xxxxxx
# 🎯 Ziel: xxxxxx

In [ ]:
# =====================================
# Schritt 5: Analyse KAQ 1: Welche Ernährungsmuster lassen sich aus individuellen Makronährstoffprofilen ableiten – und welche Krankheitsrisiken sind damit verbunden?
# Methode: KMeans-Clusteranalyse auf Protein, Fett, Zucker, Kalorien, etc.
# Ziel: Ernährungstypen identifizieren
# Zusatzanalyse: Krankheitsverteilung je Cluster
# Visualisierung: Clusterplot, Balkendiagramm Krankheitsprävalenz
# =====================================

# 🧠 Frage: xxxxxx
# 🎯 Ziel: xxxxxx

In [ ]:
# =====================================
# Schritt 5: Analyse KAQ
# =====================================

# 🧠 Frage: xxxxxx
# 🎯 Ziel: xxxxxx

In [ ]:
# =====================================
# Schritt 5: Analyse KAQ
# =====================================

# 🧠 Frage: xxxxxx
# 🎯 Ziel: xxxxxx

In [ ]:
#  In diesem Schritt bereinigen wir die df_meals-Tabelle:

# Leere Zellen prüfen, Doppelte Einträge entfernen, Datentypen prüfen
df_meals.isnull().sum()
df_meals.drop_duplicates(subset=[col for col in df_meals.columns if col != "Disease_List"], inplace=True)
df_meals.dtypes

# Disease-Spalte vorbereiten für Analyse
df_meals["Disease_List"] = df_meals["Disease"].fillna("").apply(lambda x: [d.strip() for d in x.split(",") if d.strip() != ""])

# Neu: Zähle Krankheiten je Person (0 bis n Krankheiten)
df_meals["Disease_Count"] = df_meals["Disease_List"].apply(len)

# Optional: Disease als Tuple zur Gruppierung (nur wenn .groupby() oder .value_counts() verwendet werden soll)
df_meals["Disease_Tuple"] = df_meals["Disease_List"].apply(lambda x: tuple(x))



In [ ]:
# =====================================
# Validierung: Datenqualität & Integrität
# =====================================

# 📋 Diese Zelle überprüft automatisch zentrale Qualitätsaspekte:
# - Gibt es noch leere Zellen?
# - Enthält jede Spalte den erwarteten Datentyp?
# - Stimmen Anzahl Zeilen mit anderen Tabellen überein?

# Fehlende Werte checken
missing = df_meals.isnull().sum()
print("\n# Fehlende Werte pro Spalte:")
print(missing[missing > 0])

# Erwartete Datentypen
expected_types = {
    "Ages": np.int64,
    "Gender": object,
    "Calories": np.int64,
    "Disease_List": list,
    "Disease_Tuple": tuple,
    "Disease_Count": np.int64
}

# Vergleich Ist vs. Erwartet
print("\n# Typvalidierung:")
for col, expected_type in expected_types.items():
    actual_type = df_meals[col].iloc[0].__class__
    if actual_type != expected_type:
        print(f"Typabweichung in '{col}': erwartet {expected_type}, ist {actual_type}")

# Anzahl Zeilen prüfen
print("\n# Anzahl Datensätze:", len(df_meals))


In [ ]:
# Zeige ALLE Zeilen mit fehlenden Werten in 'Breakfast Carbohydrates'
fehlende = df_meals[df_meals["Breakfast Carbohydrates"].isnull()]
pd.set_option('display.max_columns', None)  # zeigt ALLE Spalten an
display(fehlende)


In [ ]:
#Entfernen der Zeilen mit fehlenden Werten in 'Breakfast Carbohydrates'
df_meals = df_meals.dropna(subset=["Breakfast Carbohydrates"])

In [ ]:
# =====================================
# Schritt 3: Explorative Analyse nach IBCS
# =====================================

# 🧠 Frage: Unterscheiden sich kranke und gesunde Personen im Zuckerkonsum?
# 🎯 Ziel: Visuelle Verteilung und Mittelwertunterschied zeigen

sns.boxplot(data=df_meals, x=df_meals["Disease_Count"] > 0, y="Sugar")
plt.title("Zuckerzufuhr bei Personen mit und ohne Krankheit")
plt.xlabel("Krankheit vorhanden?")
plt.ylabel("Zucker (g pro Tag)")
plt.grid(True)
plt.show()

# SAY-Prinzip (Kommentar zur Grafik)
print("Personen mit Krankheit weisen im Median einen leicht höheren Zuckerkonsum auf.")


In [ ]:
# 📦 Importiere notwendige Module
import os
from nbformat import read
from nbformat.validator import validate

# 📍 Definiere den vollständigen Pfad zu deinem Notebook
notebook_path = r"onlyAnalysis_Ernährung.ipynb"

# 🧪 Überprüfe, ob die Datei existiert
if not os.path.exists(notebook_path):
    print(f"⚠️ Datei nicht gefunden: {notebook_path}")
    print("Bitte überprüfen Sie den Pfad und stellen Sie sicher, dass die Datei existiert.")
    # Exit the script gracefully
    import sys
    sys.exit(1)
    # 🗂️ Zeige den absoluten Pfad der Datei
    absolute_path = os.path.abspath(notebook_path)
    print(f"📂 Absoluter Pfad der Datei: {absolute_path}")
# 📖 Öffne das Notebook und lade den Inhalt
with open(notebook_path, "r", encoding="utf-8") as f:
    nb = read(f, as_version=4)

# ✅ Validierung durchführen
validate(nb)
print("✅ Das Notebook ist gültig und im richtigen Format!")
